# Session 1: Introduction to Quantum Optimization


# Quadratic Programs, QUBOs, and Ising Hamiltonians

Qiskit introduces the `QuadraticProgram` class to make a model of an optimization problem.
More precisely, it deals with quadratically constrained quadratic programs given as follows:


$$
\min_{x \in X} \, x^T A x + b^T x + c \\
\,\\
\text{subject to}\\
x^T A_i x + b_i^T x + c_i \leq 0, \quad i=1, \ldots, r,\\
\,\\
\text{where}\\
X = \mathbb{R}^n \times \mathbb{Z}^m \times \{0, 1\}^k\\
A \in \mathbb{R}^{(n+m+k) \times (n+m+k)}\\
b \in \mathbb{R}^{(n+m+k)}\\
c \in \mathbb{R}
$$

In addition to "$\leq$" constraints 'QuadraticProgram' also supports "$\geq$" and "$=$".

Here, we will see how to instantiate such a program using [DOcplex](https://ibmdecisionoptimization.github.io/docplex-doc/mp/index.html), injest it into Qiskit's optimization, map it to a QUBO, and then map it to an Ising Hamiltonian.

In [ ]:
# Some standard code imports
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np

# For drawing graphs
import networkx as nx

# Qiskit imports
from qiskit import Aer, execute, QuantumCircuit
from qiskit.quantum_info import Statevector

# auxilliary function to plot graphs
def plot_result(G, x):
    colors = ['r' if x[i] == 0 else 'b' for i in range(n)]
    pos, default_axes = nx.spring_layout(G), plt.axes(frameon=True)
    nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, pos=pos)

## Setting up the problem via DOcplex: MaxCut on a graph

Let's make a graph on which we will solve the [MaxCut problem](https://en.wikipedia.org/wiki/Maximum_cut).

In [ ]:
# Create graph
G = nx.Graph()

# Add 5 nodes
n = 5
G.add_nodes_from(range(n))

# Add edges: tuple is (i,j,weight) where (i,j) is the edge
edges = [(0, 1, 1.0), (0, 2, 1.0), (0, 3, 1.0), (1, 2, 1.0), (2, 3, 1.0), (2, 4, 1.0), (3, 4, 1.0)]
G.add_weighted_edges_from(edges)

# Plot graph
plot_result(G, [0]*n)

Recall that the objective function for MaxCut on a graph looks like


$$
\max_{x \in \{0, 1\}^n} \sum_{(j,k) \in E} w_{jk} (x_j + x_k - 2 x_j x_k)
$$

where $E$ denotes the set of edges, $w_{jk}$ denotes the weight of the edge going from node $j$ to node $k$, and $x_{j}$ is a binary variable indicating which partition node $j$ is located in.

In [ ]:
# Import a model from DOcplex
from docplex.mp.model import Model

# Name the model
mdl = Model('MaxCut')

# Add a binary variable to the model for each node in the graph
x = mdl.binary_var_list('x{}'.format(i) for i in range(n))

# Define the objective function
objective = mdl.sum([ w * (x[i] + x[j] - 2*x[i]*x[j]) for (i, j, w) in edges])

# And let's maximize it!
mdl.maximize(objective)

# Let's print the model
mdl.prettyprint()

Right now, our model is unconstrained. We can easily add some constraints of the form

$$
\sum_{j=0}^{n-1} x_j = b\\
\sum_{j=0}^{n-1} x_j \geq b\\
\sum_{j=0}^{n-1} x_j \leq b
$$

In [ ]:
# Add an equality constraint
b = 2
mdl.add_constraint(mdl.sum(x) == b)

# Let's print the model
mdl.prettyprint()

## Mapping to a `Quadratic Program`

As setup, you need to import the following module.

In [ ]:
from qiskit.optimization import QuadraticProgram

In [ ]:
# Instantiate an empty QuadraticProgram object
qp = QuadraticProgram()

# Put the model inside it
qp.from_docplex(mdl)

In [ ]:
# Let's print the model!
# Note that in the LP format the quadratic part has to be scaled by a factor $1/2$.
# Thus, when printing as LP format, the quadratic part is first multiplied by 2 and then divided by 2 again.
print(qp.export_as_lp_string())

The `QuadraticProgram` supports three types of variables:
- Binary variable
- Integer variable
- Continuous variable

When you add variables, you can specify names, types, lower bounds and upper bounds.

When you display your problem as LP format,
`Binaries` denotes binary variables and `Generals` denotes integer variables.
If variables are not included in either `Binaries` or `Generals`, such variables are continuous ones with default lower bound = 0 and upper bound = infinity.
Note that you cannot use 'e' or 'E' as the first character of names due to the [specification of LP format](https://www.ibm.com/support/knowledgecenter/SSSA5P_12.7.1/ilog.odms.cplex.help/CPLEX/FileFormats/topics/LP_VariableNames.html).

You can access the constant, the linear term, and the quadratic term by looking at `Quadratic.objective.{constant, linear, quadratic}`, respectively.
As for linear and quadratic terms, you can get a dense matrix (`to_array`), a sparse matrix (`coefficients`), and a dictionary (`to_dict`).
For dictionaries, you can specify whether to use variable indices or names as keys.

In [ ]:
# Look at the coefficients of the linear term of the QP as dictionary
qp.objective.linear.to_dict()

In [ ]:
# Look at the coefficients of the quadratic term of the QP as dictionary
qp.objective.quadratic.to_dict()

## Solve using a classical optimizer

Since this problem is pretty small, we can go ahead and solve it directly.

Note: if you wanted to solve this using CPlex, you'd need to install that as well.

In [ ]:
# Import the Numpy Solver
from qiskit.aqua.algorithms import NumPyMinimumEigensolver

# Import the MinEigenOptimizer algorithm
from qiskit.optimization.algorithms import MinimumEigenOptimizer

In [ ]:
solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())

In [ ]:
# Solve Quadratic Program
result = solver.solve(qp)
print(result)
plot_result(G, result.x)

## Convert the QuadraticProgram to a QUBO

Recall that the objective function for a QUBO looks like

$$
\min_{x \in \{0, 1\}^k} \, x^T A x + c \\
$$

Our objective function doesn't look like that, so we need to do some conversions. Qiskit's optimization module helps us do so. In general, optimization algorithms are defined for a certain formulation of a quadratic program and we need to convert our problem to the right type.

To map a problem to the correct input format, the optimization module of Qiskit offers a variety of converters. Currently, Qiskit contains the following converters.
- `InequalityToEquality`: converts inequality constraints into equality constraints with additional slack variables.
- `IntegerToBinary`: converts integer variables into binary variables and corresponding coefficients. 
- `LinearEqualityToPenalty`: convert equality constraints into additional terms of the object function.
- `QuadraticProgramToQubo`: a wrapper for `InequalityToEquality`, `IntegerToBinary`, and `LinearEqualityToPenalty` for convenience.

In [ ]:
# Re-print the model to see what we're working with here.
print(qp.export_as_lp_string())

We've got a linear equality constraint, so we need that converter.

In [ ]:
from qiskit.optimization.converters import LinearEqualityToPenalty


`LinearEqualityToPenalty` converts linear equality constraints into additional quadratic penalty terms of the objective function to map `QuadraticProgram` to an unconstrained form.
An input to the converter has to be a `QuadraticProgram` with only linear equality constraints. Those equality constraints, e.g. $\sum_i a_i x_i  = b$ where $a_i$ and $b$ are numbers and $x_i$ is a variable, will be added to the objective function in the form of $M(b - \sum_i a_i x_i)^2$ where $M$ is a large number as penalty factor. 
By default $M= 1e5$. The sign of the term depends on whether the problem type is a maximization or minimization.

In [ ]:
# Instantiate the LinearEqualityToPenalty object,
# and do the conversion
# Note what happens as you change the penalty
qp_eq = LinearEqualityToPenalty(penalty=10).convert(qp)

In [ ]:
print(qp_eq.export_as_lp_string())

In [ ]:
# Solve this quadratic program
result = solver.solve(qp_eq)
print(result)
plot_result(G, result.x)

For quadratic programs with more complex constraints, Qiskit's optimization module provides the `QuadraticProgramToQubo` function to directly map from a QP to a QUBO.

In [ ]:
from qiskit.optimization.converters import QuadraticProgramToQubo

In [ ]:
# Instantiate the QuadraticProgramToQubo object, and do the conversion
# Put in the penalty for the LinearEqualityToConstraint converter
direct_translation = QuadraticProgramToQubo(penalty=10).convert(qp)

# Let's print the model!
print(direct_translation.export_as_lp_string())

## Mapping a QUBO to an Ising Hamiltonian

Consider the QUBO
$$
\min_{x \in \{0, 1\}^k} \, x^T A x + c.
$$

To map onto an Ising Hamiltonian, we do the following:
1. Substitute<br>
$x_i = (1 - z_i)/2,$<br>
where $z_i \in \{-1, +1\}$.
<br>

2. Replace<br>
$z_i z_j = \sigma_Z^i \otimes \sigma_Z^j$, and<br>
$z_i = \sigma_Z^i$,<br>
where $\sigma_Z^i$ denotes the Pauli Z-matrix $\left(\begin{array}{cc}1&0\\0&-1\end{array}\right)$ on the <font color="blue">$i$-th qubit</font>.

Qiskit Aqua provides abstractions and representations of quantum states and operators that are useful for this conversion.

In [ ]:
# import matrizes I, Z and states |0>, |1>
from qiskit.aqua.operators import I, Z, Zero, One, StateFn

We can turn these abstract representations into matrices.

In [ ]:
print('I =   \n', I.to_matrix())
print('Z =   \n', Z.to_matrix())
print('|0> = \n', Zero.to_matrix())
print('|1> = \n', One.to_matrix())

We can evaluate expectation values as follows

In [ ]:
print('<0|Z|0> =', (~StateFn(Z) @ Zero).eval())
print('<1|Z|1> =', (~StateFn(Z) @ One).eval())

Tensor products are formed using the `^` method:

In [ ]:
print('ZZ = \n', (Z ^ Z).to_matrix())
print()
print('|0>|0> =', (Zero^Zero).to_matrix())
print('|0>|1> =', (Zero^One).to_matrix())
print('|1>|0> =', (One^Zero).to_matrix())
print('|1>|1> =', (One^One).to_matrix())

And we can compute expectation values of these tensor products as well.

In [ ]:
print('<00|ZZ|00> =', (~StateFn(Z^Z) @ (One^One)).eval())
print('<01|ZZ|01> =', (~StateFn(Z^Z) @ (One^Zero)).eval())
print('<10|ZZ|10> =', (~StateFn(Z^Z) @ (Zero^One)).eval())
print('<11|ZZ|11> =', (~StateFn(Z^Z) @ (One^One)).eval())

If we had a Hamiltonian and some particular state, we can compute the expectation value of that state's energy with respect to the Hamiltonian in the same way.

In [ ]:
# define Hamiltonian
H = I ^ I ^ Z ^ Z ^ I

# define state
psi = Zero ^ One ^ Zero ^ One ^ Zero

# evaluate expected value
print('<psi|H|psi> =', (~StateFn(H) @ psi).eval())

Let's convert the quadratic program (with the equality constraint removed) to an Ising Hamiltonian.

In [ ]:
H, offset = qp_eq.to_ising()

In [ ]:
# The offset is how much we have to shift the answer we get from a minimum energy optimizer.
# It arises from the coefficient of the Hamiltonian where all the operations on qubits are the identity.
print('offset =', offset)


In [ ]:
# Later, we'll use different algorithms to minimize the energy of the Hamiltonian.
print('H =', H)

In [ ]:
# Note that if we tried to convert the original QP, with the constraint, Qiskit would tell us what to do!
qp.to_ising()